<a href="https://colab.research.google.com/github/shwehan/zc-datatalks/blob/main/deployment_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 66 bytes


In [ ]:
!kaggle datasets download -d kapturovalexander/bank-credit-scoring

  0% 0.00/182k [00:00<?, ?B/s]
100% 182k/182k [00:00<00:00, 66.4MB/s]


In [ ]:
from zipfile import ZipFile
file_name = "/content/bank-credit-scoring.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
df = pd.read_csv(r"/content/bank.csv", sep=';').drop_duplicates()

In [ ]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [ ]:
pip --version

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
!pip install flask-ngrok
!ngrok authtoken '2WmJu9a0XTZZQGp3dYYt2ohUQGR_57Ey26Z8C3KxcGrQV8U18'

/bin/bash: line 1: ngrok: command not found


Q3: Write a script to LOAD these models with pickle

Score this client

In [ ]:
url = f"https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework"

!wget $url/model1.bin
!wget $url/dv.bin

--2023-10-15 03:31:02--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 842 [application/octet-stream]
Saving to: ‘model1.bin’

model1.bin          100%[===================>]     842  --.-KB/s    in 0s      

2023-10-15 03:31:03 (27.1 MB/s) - ‘model1.bin’ saved [842/842]

--2023-10-15 03:31:03--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2023/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... con

In [ ]:
df.y.head()

0    no
1    no
2    no
3    no
4    no
Name: y, dtype: object

In [ ]:
features = ['job','duration', 'poutcome']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X, df.y)

In [ ]:
y_pred = model.predict_proba(X)[:, 1]


In [ ]:
C = 1.0
n_splits = 5

In [ ]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []


for train_idx, val_idx in kfold.split(df):
    df_train = df.iloc[train_idx]
    df_val = df.iloc[val_idx]

    y_train = df.y.values
    y_val = df_val.y.values

    model = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X, df.y)
    y_pred = model.predict_proba(X)[:, 1]

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

ValueError: ignored

In [ ]:
import pickle

output_file = f'model_C=model1.bin'
output_file

'model_C=model1.bin'

In [ ]:
f_out = open(output_file, 'wb')
pickle.dump((dv, model, f_out))
f_out.close()

TypeError: ignored